In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import math
import warnings
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot
from scipy.stats import uniform, randint
warnings.filterwarnings('ignore')

# Model and performance evaluation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from scikeras.wrappers import KerasClassifier
from joblib import dump, load
from sklearn.svm import SVC

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import auc, average_precision_score, precision_recall_curve

# Hyperparameter tuninghCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval
from xgboost import plot_importance

# Data imputation
from imblearn.over_sampling import SMOTE


## Functions

#### 1) Evaluate model

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)

    roc_auc = roc_auc_score(y_test, pred_proba)
    precision_, recall_, thr = precision_recall_curve(y_test, pred_proba)
    pr_auc = auc(recall_, precision_)

    result = '{0:.4f}, {1:.4f}, {2:.4f}, {3:.4f}, {4:.4f}, {5:.4f}'.format(accuracy, precision, recall, f1, roc_auc, pr_auc)

    return confusion, result


#### 2) Plot AUROC

In [ ]:
def roc_curve_plot(y_test, opted_predict_prob,label_name):
    fpr, tpr, thresholds = roc_curve(y_test, opted_predict_prob)

    plt.plot(fpr, tpr, label = label_name)
    plt.plot([0,1], 'k--')
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title("AUROC")
    plt.legend(fontsize=6)

#### 3) Plot AUPRC

In [ ]:
def plot_pr_curve(recall, precision,label_name):
    pyplot.plot([0, 1], 'k--')
    pyplot.plot(recall, precision, label=label_name)

    pyplot.xlabel('Recall')
    pyplot.ylabel('Precision')
    plt.title("AUPRC")
    pyplot.legend(fontsize=6)

#### 4) Preprocess_data

In [ ]:
def preprocess_data(input, output):

    features = pd.read_excel(input)
    label = pd.read_excel(output)

    X_train, X_test, y_train, y_test = train_test_split(features, label,
                                                    test_size=0.2, random_state=1)


    y_train = np.ravel(y_train)
    y_test = np.ravel(y_test)

    # imputating train_data
    smote = SMOTE(random_state=0)
    X_train, y_train = smote.fit_resample(X_train, y_train)

    return X_train, X_test, y_train, y_test

#### 5) Plot_curve

In [ ]:
def plot_curve(input, output, curve, y_test, opted_predict_prob):

    # set label
    in_ = re.sub(r'[^0-9]', '', input)
    out_ = re.sub(r'[^0-9]', '', output)
    in_ = int(in_)

    # plot ROCAUC
    if curve == 'roc':
        roc_auc = round(roc_auc_score(y_test, opted_predict_prob), 3)

        label = f"{in_-2}~{in_} Hours since ICU admission, {curve.upper()}_ROCAUC: {roc_auc}"
        roc_curve_plot(y_test, opted_predict_prob, label_name = label)

    # plot PRAUC
    elif curve == 'pr':
        precision, recall, thresholds = precision_recall_curve(y_test, opted_predict_prob)
        pr_auc = round(auc(recall, precision),3)
        label = f"{in_-2}~{in_} Hours since ICU admission, {curve.upper()}_PRAUC: {pr_auc}"
        plot_pr_curve(recall, precision, label_name=label)

In [ ]:
def plot_curve_(input, output, curve, y_test, opted_predict_prob, model_name):

    # set label
    in_ = re.sub(r'[^0-9]', '', input)
    out_ = re.sub(r'[^0-9]', '', output)

    # plot plot ROCAUC
    if curve == 'roc':
        roc_auc = round(roc_auc_score(y_test, opted_predict_prob), 3)
        label = f"{model_name}, {curve.upper()}_ROCAUC: {roc_auc}"
        roc_curve_plot(y_test, opted_predict_prob, label_name = label)

    # plot PRAUC
    elif curve == 'pr':
        precision, recall, thresholds = precision_recall_curve(y_test, opted_predict_prob)
        pr_auc = round(auc(recall, precision),3)
        label = f"{model_name}, {curve.upper()}_PRAUC: {pr_auc}"
        plot_pr_curve(recall, precision, label_name=label)

#### 6) Train & Calibrate

In [ ]:
def train_and_calibrate_model(name, model, param, X_train, y_train, X_test, y_test, input, output):
    in_ = re.sub(r'[^0-9]', '', input)
    out_ = re.sub(r'[^0-9]', '', output)

    # random search
    scoring = ['roc_auc']

    kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)

    random_search = RandomizedSearchCV(estimator=model,
                           param_distributions=param,
                           n_iter=48,
                           scoring=scoring,
                           refit='roc_auc',
                           n_jobs=-1,
                           cv=kfold,
                           verbose=0)

    random_result = random_search.fit(X_train, y_train)

    # save dump file
    dump(random_search, f'./trained/{name}_matrix_trained_{in_}hr_{out_}hr.joblib')
    print(f"save matrix: {name}_matrix_trained_{in_}hr_{out_}hr.joblib")

    opted_predict = random_search.predict(X_test)
    opted_predict_prob = random_search.predict_proba(X_test)[:,1]

    return opted_predict, opted_predict_prob

#### 7) Execute

In [ ]:
def execute(dir_input, dir_output, n_fold, imputation, curve, name):
    i = 0
    total = len(dir_input)*len(dir_output)
    df_result = pd.DataFrame(columns = ["Accuracy", "Precision", "Recall", "F1", "ROC AUC", "PR AUC"])

    models = {'LR': LR_model,
                      'DT': DT_model,
                      'RF': RF_model,
                      'XGB': XGB_model,
                      'KNN': KNN_model,
                      'ANN': ANN_model,
                      'SVM': SVM_model
             }

    model = models[name]

    for output in dir_output:
        out_ = re.sub(r'[^0-9]', '', output)
        idx = {'Accuracy': f'{out_}hr'}
        df_result = df_result.append(idx, ignore_index=True)

        for input in dir_input:

            in_ = re.sub(r'[^0-9]', '', input)
            print(f"{i/total*100}% processing..")

            ## Run
            confusion, result = model(name, input, output, n_fold, curve, 1)

            ## post-process results from model
            dic = {}
            temp = result.split(', ')

            index = 0
            for key in df_result.keys():
                dic[key] = temp[index]
                index += 1

            df_result = df_result.append(dic, ignore_index=True)
            plt.savefig(f'./result_graph/{name}_{imputation}_{curve}_curve_{out_}hr.png', dpi=300)
            i = i + 1

        plt.cla()

        # export
        df_result.to_csv(f'./result_csv/result_{name}_{imputation}_prolonged.csv')


In [ ]:
## 모델별 비교 목적
def execute_(input_path, output_path, n_fold, imputation, curve, list_models):

    models = {'LR': LR_model,
                      'DT': DT_model,
                      'RF': RF_model,
                      'KNN': KNN_model,
                      'XGB': XGB_model,
                      'SVM': SVM_model
             }


    in_ = re.sub(r'[^0-9]', '', input_path)
    out_ = re.sub(r'[^0-9]', '', output_path)

    if 'ANN' in list_models:
      ANN_model(input_path, output_path, n_fold, curve)
      plt.savefig(f'./result_graph/ALL_{imputation}_{curve}_curve_{in_}hr.png', dpi=300)
      models.pop(models.index('ANN'))

    total = len(list_models)

    i = 0
    for name in list_models:
        model = models[name]

        print(f"{i/total*100}% processing..")

        ## Run
        confusion, result = model(name, input_path, output_path, n_fold, curve, 2)

        plt.savefig(f'./result_graph/ALL_{imputation}_{curve}_curve_{in_}hr_{out_}hr.png', dpi=300)

        i = i + 1

## Models

#### 1) Logistic Regeression

In [ ]:
def LR_model(name, input, output, n_fold, curve, mode):

    X_train, X_test, y_train, y_test = preprocess_data(input, output)

    ## model
    model = LogisticRegression(random_state=1, max_iter=2000)

    ## optimizing hyper-param for model
    param = {
        'penalty': ['l1','l2'],
        'dual': [True, False],
        'tol': [0.0001],
        'C': [1],
        'fit_intercept': [True, False],
        'intercept_scaling': [1],
        'class_weight': [1, 'balanced'],
        'random_state': [0],
        'solver': ['newton-cg','lbfgs','liblinear','sag','saga'],
        'max_iter': [100],
        'multi_class': ['ovr','multinomial','auto'],
        'verbose': [0],
        'warm_start': [True, False],
        'n_jobs': [None],
        'l1_ratio': [None]
    }

    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, param, X_train, y_train, X_test, y_test, input, output)

    ## Ploting curve
    if mode = 1:
      plot_curve(input, output, curve, y_test, opted_predict_prob)
    elif mode = 2:
      plot_curve_(input, output, curve, y_test, opted_predict_prob, name)

    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


#### 2) Decision Tree

In [ ]:
def DT_model(name, input, output, n_fold, curve, mode):

    X_train, X_test, y_train, y_test = preprocess_data(input, output)

    ## model
    model = DecisionTreeClassifier(random_state=1)

    ## optimizing hyper-param for model
    param = {
         'min_impurity_decrease': [0.001, 0.01, 0.001],
         'max_depth': range(5, 20, 1),
         'min_samples_split': range(2, 100, 10)
    }

    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, param, X_train, y_train, X_test, y_test, input, output)

    ## Ploting curve
    if mode = 1:
      plot_curve(input, output, curve, y_test, opted_predict_prob)
    elif mode = 2:
      plot_curve_(input, output, curve, y_test, opted_predict_prob, name)

    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


#### 3) Random Forest

In [ ]:
def RF_model(name, input, output, n_fold, curve, mode):

    X_train, X_test, y_train, y_test = preprocess_data(input, output)

    ## model
    model = RandomForestClassifier(n_jobs=-1, random_state=1)

    param = {
        'n_estimators' : [10, 100],
        'max_depth' : [6, 8, 10, 12],
        'min_samples_leaf' : [8, 12, 18],
        'min_samples_split' : [8, 16, 20]
    }

    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, param, X_train, y_train, X_test, y_test, input, output)

    ## Ploting curve
    if mode = 1:
      plot_curve(input, output, curve, y_test, opted_predict_prob)
    elif mode = 2:
      plot_curve_(input, output, curve, y_test, opted_predict_prob, name)


    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


#### 4) KNN

In [ ]:
def KNN_model(name, input, output, n_fold, curve, mode):

    X_train, X_test, y_train, y_test = preprocess_data(input, output)

    ## model
    model = KNeighborsClassifier(n_neighbors=5)

    param = {
        'n_neighbors' : list(range(1,30)),
        'weights' : ["uniform", "distance"],
        'metric' : ['euclidean', 'manhattan', 'minkowski'],
        'leaf_size' : range(1, 50, 5)
    }

    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, param, X_train, y_train, X_test, y_test, input, output)

    ## Ploting curve
    if mode = 1:
      plot_curve(input, output, curve, y_test, opted_predict_prob)
    elif mode = 2:
      plot_curve_(input, output, curve, y_test, opted_predict_prob, name)

    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


#### 5) XGBoost

In [ ]:
def XGB_model(name, input, output, n_fold, curve, mode):

    X_train, X_test, y_train, y_test = preprocess_data(input, output)

    ## model
    model = XGBClassifier(booster='gbtree', objective='binary:logistic')

    param = {
        'n_estimators': [100,200,400,800,1000],
        'learning_rate': [0.01,0.05,0.1,0.2,0.3,0.4,0.5],
        'gamma': [0,0.01,0.1,0.5,1,2],
        'min_child_weight':[1,2,3,4,5]
    }

    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, param, X_train, y_train, X_test, y_test, input, output)

    ## Ploting curve
    if mode = 1:
      plot_curve(input, output, curve, y_test, opted_predict_prob)
    elif mode = 2:
      plot_curve_(input, output, curve, y_test, opted_predict_prob, name)

    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


#### 6) ANN

In [ ]:
def ANN_model(input, output, n_fold, curve):

    ## processing data for training/test
    features = pd.read_excel(input)
    label = pd.read_excel(output)

    len_features = int(features.shape[1])

    X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)

    y_train = np.ravel(y_train)
    y_test = np.ravel(y_test)

    smote = SMOTE(random_state=0)
    X_train, y_train = smote.fit_resample(X_train, y_train)


    ## model
    model = keras.Sequential()

    model.add(keras.layers.InputLayer(input_shape=(len_features,)))

    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model = KerasClassifier(model)

    ## optimizing hyper-param for model

    # random search
    param = {
        'batch_size': [16, 32, 64, 128],
        'epochs': [10, 20]
    }

    scoring = ['accuracy']

    kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=1)

    random_search = RandomizedSearchCV(estimator=model,
                                   param_distributions=param,
                                   scoring=scoring,
                                   cv=kfold,
                                   refit='accuracy',
                                   n_jobs=-1,
                                   verbose=0)

    random_result = random_search.fit(X_train, y_train)

    best_model = random_search.best_estimator_
    opted_predict = best_model.predict(X_test)
    opted_predict_prob = random_search.predict_proba(X_test)[:,1]

    ## Ploting curve
    plot_curve(input, output, curve, y_test, opted_predict_prob)

    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


#### 7) SVM

In [ ]:
def SVM_model(name, input, output, n_fold, curve, mode):

    ## processing data for training/test
    X_train, X_test, y_train, y_test = preprocess_data(input, output)


    ## model
    model = SVC(random_state=0, probability=True)

    param = {
        'C': [0.1,1, 10, 100],
        'kernel': ['poly','sigmoid','linear','rbf'],
        'gamma': [1,0.1,0.01,0.001]
    }

    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, param, X_train, y_train, X_test, y_test, input, output)

    ## Ploting curve
    if mode = 1:
      plot_curve(input, output, curve, y_test, opted_predict_prob)
    elif mode = 2:
      plot_curve_(input, output, curve, y_test, opted_predict_prob, name)

    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)


    return temp_comfusion, temp_result

## Execute model

#### 1) Execute model performance by prediction time windows

In [ ]:
#### configure
dir_input = [
    './df_input/df_2hr.xlsx',
    './df_input/df_4hr.xlsx',
    './df_input/df_6hr.xlsx',
    './df_input/df_8hr.xlsx',
    './df_input/df_10hr.xlsx',
    './df_input/df_12hr.xlsx'
]

dir_output = ['./df_output/df_categorical_output_hr/df_categorical_output_60hr.xlsx']

n_fold = 5
imputation = 'smote'
curves = ['roc','pr']

In [ ]:
## LR, DT, RF, XGB, KNN, SVM
## Performance comparison by prediction time windows

model = ['LR', 'DT', 'RF', 'XGB', 'KNN', 'SVM']

for curve in curves:
    execute(dir_input, dir_output, n_fold, imputation, curve, model)
    plt.cla()

In [ ]:
## ANN

curves = ['roc','pr']

for curve in curves:
    i = 0
    total = len(dir_input)*len(dir_output)
    df_result = pd.DataFrame(columns = ["Accuracy", "Precision", "Recall", "F1", "ROC AUC", "PR AUC"])

    for output in dir_output:
        out_ = re.sub(r'[^0-9]', '', output)
        idx = {'Accuracy': f'{out_}hr'}
        df_result = df_result.append(idx, ignore_index=True)

        for input in dir_input:

            in_ = re.sub(r'[^0-9]', '', input)
            print(f"{i/total*100}% processing..")


            ## Run
            confusion, result = ANN_model(input, output, n_fold, curve)

            ## post-process results from model
            dic = {}
            temp = result.split(', ')

            index = 0
            for key in df_result.keys():
                dic[key] = temp[index]
                index += 1

            df_result = df_result.append(dic, ignore_index=True)
            plt.savefig(f'./result_graph/ANN_{imputation}_{curve}_curve_{out_}hr.png', dpi=300)
            i = i + 1

        plt.cla()

    # export
    df_result.to_csv(f'./result_csv/result_ANN__{imputation}_prolonged.csv')

#### 2) Execute model performances by prediction models

In [ ]:
# Performance comparison by prediction model

dir_input = './df_input/df_12hr.xlsx'
dir_output = './df_output/df_categorical_output_hr/df_categorical_output_60hr.xlsx'

models = ['DT','KNN','LR','RF','SVM','XGB', 'ANN']
curves = ['roc', 'pr']
n_fold = 5
imputation = 'smote'

for curve in curves:
    execute_(dir_input, dir_output, n_fold, imputation, curve, models)
    plt.cla()
